In [ ]:
import os
from dotenv import load_dotenv
# load_dotenv()
# api_key = os.getenv("GEMINI_API_KEY")
# api_key

In [ ]:
from IPython.display import display, Markdown
import textwrap
def to_markdown(text):
    display(Markdown(textwrap.dedent(text)))

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")
name = llm("What is the meaning of life?")
print(name)

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=['cusine'],
    template= "I want to open a restaurant that serves {cusine} food. Give me a fancy headline of the restaurant."
)
prompt.format(cusine = 'Indian')

In [ ]:
from langchain.chains import LLMChain
headline_chain = LLMChain(llm=llm, prompt=prompt)
headline_chain.run('Indian')

In [ ]:
prompt = PromptTemplate(
    input_variables=['headline'],
    template= "Suggest menu items for the restaurant with the headline: {headline}"
)
menu_chain = LLMChain(llm=llm, prompt=prompt)
to_markdown(menu_chain.run('The Saffron Palace: Culinary Delights from the Heart of India'))

In [ ]:
from langchain.chains import SimpleSequentialChain
#The output of the first chain will be the input to the second chain
chain = SimpleSequentialChain(chains = [headline_chain, menu_chain])
to_markdown(chain.run('Indian'))

In [47]:
prompt_headline = PromptTemplate(
    input_variables=['cusine'],
    template= "I want to open a restaurant that serves {cusine} food. Give me a fancy headline of the restaurant."
)
headline_chain = LLMChain(llm=llm, prompt=prompt_headline, output_key='headline')

prompt_menu = PromptTemplate(
    input_variables=['headline'],
    template= "Suggest little menu items for the restaurant with the headline: {headline}"
)
menu_chain = LLMChain(llm=llm, prompt=prompt_menu, output_key='menu')

In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains = [headline_chain, menu_chain],
    input_variables = ['cusine'],
    output_variables = ['headline', 'menu']
)
print(chain('Indian')['menu'])

In [35]:
from langchain.agents import AgentType, initialize_agent, load_tools
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("In a right angled triangle, the length of the hypotenuse is 5 and the length of one of the sides is 3. What is the length of the other side?")



> Entering new AgentExecutor chain...
I need to use the Pythagorean theorem to find the length of the other side.
Action: Calculator
Action Input: 5**2 - 3**2
Observation: Answer: 16
Thought:I now know the final answer
Final Answer: 4

> Finished chain.


'4'

In [48]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
chain.run('Indian')

'**Appetizers**\n\n* Samosas (Crispy pastries filled with spiced potatoes, onions, and peas)\n* Pakoras (Vegetable fritters dipped in chickpea batter)\n* Onion Bhajis (Onion rings coated in chickpea batter)\n* Papadum (Thin, crispy lentil crackers)\n\n**Main Courses**\n\n* Butter Chicken (Tender chicken in a creamy tomato sauce)\n* Tikka Masala (Chicken or paneer marinated in yogurt, spices, and grilled, then cooked in a creamy tomato sauce)\n* Palak Paneer (Spinach and paneer cheese cooked in a creamy sauce)\n* Chana Masala (Chickpeas cooked in a spiced tomato sauce)\n\n**Sides**\n\n* Dal Makhani (Black lentil soup)\n* Jeera Rice (Rice cooked with cumin and ghee)\n* Naan (Soft, fluffy flatbread)\n* Raita (Cucumber and yogurt sauce)\n\n**Desserts**\n\n* Gulab Jamun (Sweet dumplings made from milk solids and flour, soaked in sugar syrup)\n* Ras Malai (Sweet dumplings made from milk solids and flour, soaked in cream and flavored with cardamom)\n* Kheer (Rice pudding)\n* Kulfi (Frozen des

In [49]:
chain.run('Italian')

'**Antipasti**\n\n* Caprese skewers with fresh mozzarella, tomatoes, and basil\n* Bruschetta topped with roasted tomatoes, garlic, and basil\n* Calamari fritti with marinara sauce\n\n**Insalate**\n\n* Caesar salad with romaine lettuce, croutons, Parmesan cheese, and Caesar dressing\n* Caprese salad with fresh mozzarella, tomatoes, and basil\n* Arugula salad with goat cheese, walnuts, and balsamic vinaigrette\n\n**Pasta**\n\n* Spaghetti alla carbonara with eggs, Parmesan cheese, and pancetta\n* Lasagna with layers of pasta, meat sauce, and cheese\n* Penne with vodka sauce and Italian sausage\n\n**Secondi**\n\n* Chicken Marsala with a savory Marsala wine sauce\n* Salmon piccata with a lemon-butter sauce\n* Veal parmigiana with melted mozzarella and marinara sauce\n\n**Contorni**\n\n* Roasted vegetables with olive oil and herbs\n* Garlic bread with marinara sauce\n* Mashed potatoes with butter and parsley\n\n**Dolci**\n\n* Tiramisu with espresso-soaked ladyfingers and mascarpone cream\n* 

In [50]:
print(chain.memory.buffer)

Human: Indian
AI: **Appetizers**

* Samosas (Crispy pastries filled with spiced potatoes, onions, and peas)
* Pakoras (Vegetable fritters dipped in chickpea batter)
* Onion Bhajis (Onion rings coated in chickpea batter)
* Papadum (Thin, crispy lentil crackers)

**Main Courses**

* Butter Chicken (Tender chicken in a creamy tomato sauce)
* Tikka Masala (Chicken or paneer marinated in yogurt, spices, and grilled, then cooked in a creamy tomato sauce)
* Palak Paneer (Spinach and paneer cheese cooked in a creamy sauce)
* Chana Masala (Chickpeas cooked in a spiced tomato sauce)

**Sides**

* Dal Makhani (Black lentil soup)
* Jeera Rice (Rice cooked with cumin and ghee)
* Naan (Soft, fluffy flatbread)
* Raita (Cucumber and yogurt sauce)

**Desserts**

* Gulab Jamun (Sweet dumplings made from milk solids and flour, soaked in sugar syrup)
* Ras Malai (Sweet dumplings made from milk solids and flour, soaked in cream and flavored with cardamom)
* Kheer (Rice pudding)
* Kulfi (Frozen dessert made

In [55]:
from langchain.chains import ConversationChain
convo = ConversationChain(llm=llm)
print(convo.run('Hi'))

Hello there! How can I assist you today?


In [56]:
convo.run('Who is the greates cricketer from the 20th century?')

'Sure, I can help you with that.\n \nAccording to the International Cricket Council, the greatest cricketer from the 20th century is Sir Garfield Sobers. He was a West Indian all-rounder who played from 1954 to 1974. Sobers was a gifted batsman, bowler, and fielder. He is widely regarded as one of the greatest all-rounders of all time. \n\nSobers played in 93 Test matches for the West Indies, scoring 8,032 runs at an average of 57.78. He also took 235 wickets at an average of 34.03. In addition to his Test career, Sobers also played in 160 One-Day Internationals, scoring 2,648 runs at an average of 41.47.\n\nSobers was inducted into the ICC Cricket Hall of Fame in 2009. He is also a member of the West Indies Cricket Hall of Fame.'

In [59]:
print(convo.memory.buffer)

Human: Hi
AI: Hello there! How can I assist you today?
Human: Who is the greates cricketer from the 20th century?
AI: Sure, I can help you with that.
 
According to the International Cricket Council, the greatest cricketer from the 20th century is Sir Garfield Sobers. He was a West Indian all-rounder who played from 1954 to 1974. Sobers was a gifted batsman, bowler, and fielder. He is widely regarded as one of the greatest all-rounders of all time. 

Sobers played in 93 Test matches for the West Indies, scoring 8,032 runs at an average of 57.78. He also took 235 wickets at an average of 34.03. In addition to his Test career, Sobers also played in 160 One-Day Internationals, scoring 2,648 runs at an average of 41.47.

Sobers was inducted into the ICC Cricket Hall of Fame in 2009. He is also a member of the West Indies Cricket Hall of Fame.


In [68]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=5)
 
convo = ConversationChain(llm=llm, memory=memory)
print(convo.run('Who won the first cricket world cup?'))

West Indies


In [69]:
convo.run('Who is the best cricketer in the world?')

'There is no consensus on who is the best cricketer in the world. However, some of the players who are often considered to be among the best include Virat Kohli, Steve Smith, Kane Williamson, and Joe Root.'

In [70]:
convo.run('When did that cricket world cup take place?')

'The first Cricket World Cup took place from May 30 to June 25, 1975.'

In [71]:
convo.run('Who won the second one')

'West Indies'